In [ ]:
# Импортируем необходимые библиотеки
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# Задаем параметры модели
vocab_size = 10000 # Размер словаря
embed_size = 128 # Размерность векторов слов
hidden_size = 256 # Размерность скрытого состояния LSTM
num_layers = 5 # Количество слоев LSTM
dropout = 0.2 # Вероятность дропаута
batch_size = 256 # Размер батча
seq_len = 5 # Длина последовательности слов
lr = 0.01 # Скорость обучения
epochs = 10 # Количество эпох обучения

# Создаем класс модели
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout):
        super(LSTMModel, self).__init__()
        # Слой для встраивания слов в вектора
        self.embed = nn.Embedding(vocab_size, embed_size)
        # Слой LSTM
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        # Слой для выхода
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        # Преобразуем слова в вектора
        x = self.embed(x)
        # Пропускаем вектора через LSTM
        out, h = self.lstm(x, h)
        # Преобразуем выход LSTM в логиты для каждого слова
        out = out.reshape(-1, out.size(2))
        out = self.linear(out)
        return out, h

# Создаем экземпляр модели
model = LSTMModel(vocab_size, embed_size, hidden_size, num_layers, dropout)
# Создаем функцию потерь
criterion = nn.CrossEntropyLoss()
# Создаем оптимизатор
optimizer = optim.Adam(model.parameters(), lr=lr)

# Создаем функцию для обучения модели
def train(model, criterion, optimizer, epochs):
    # Переводим модель в режим обучения
    model.train()
    # Инициализируем скрытое состояние LSTM
    h = torch.zeros(num_layers, batch_size, hidden_size)
    # Обучаем модель в течение заданного количества эпох
    for epoch in range(epochs):
        # Сбрасываем счетчик потерь
        epoch_loss = 0
        # Итерируем по батчам данных
        for x, y in data_loader:
            # Обнуляем градиенты
            optimizer.zero_grad()
            # Переводим данные в тензоры
            x = torch.LongTensor(x)
            y = torch.LongTensor(y)
            # Пропускаем данные через модель
            out, h = model(x, h)
            # Вычисляем потери
            loss = criterion(out, y.view(-1))
            # Делаем обратное распространение ошибки
            loss.backward()
            # Обновляем параметры модели
            optimizer.step()
            # Добавляем потери к счетчику
            epoch_loss += loss.item()
        # Выводим среднюю потерю за эпоху
        print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(data_loader)}")

# Создаем функцию для тестирования модели
def test(model, words):
    # Переводим модель в режим оценки
    model.eval()
    # Инициализируем скрытое состояние LSTM
    h = torch.zeros(num_layers, 1, hidden_size)
    # Преобразуем слова в индексы
    x = [word_to_index[word] for word in words]
    # Преобразуем индексы в тензор
    x = torch.LongTensor(x).unsqueeze(0)
    # Пропускаем тензор через модель
    out, h = model(x, h)
    # Получаем логит для последнего слова
    out = out[-1]
    # Преобразуем логит в вероятности
    prob = F.softmax(out, dim=0)
    # Сэмплируем слово из распределения
    index = torch.multinomial(prob, 1).item()
    # Преобразуем индекс в слово
    word = index_to_word[index]
    # Возвращаем предсказанное слово
    return word

# Загружаем данные для обучения (примерный формат)
data_loader = load_data(batch_size, seq_len)
# Создаем словари для преобразования слов в индексы и обратно
word_to_index, index_to_word = build_vocab(vocab_size)

# Обучаем модель
train(model, criterion, optimizer, epochs)

# Тестируем модель на примерах
words = ["I", "love", "to"]
predicted_word = test(model, words)
print(f"{words} -> {predicted_word}")

words = ["She", "is", "a"]
predicted_word = test(model, words)
print(f"{words} -> {predicted_word}") 